# Calculate magnitudes of the stars

Written by Evgenii N.

This code calculates the magnitudes of stars.

## Prerequisite code

In [1]:
# Import libraries that we will use later in this notebook
import os
import shutil
import ccdproc
import numpy as np
from astropy.visualization import ZScaleInterval, MinMaxInterval, ImageNormalize
from astropy import units as u
from matplotlib.colors import LogNorm
from ccdproc import CCDData
import matplotlib.pyplot as plt
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.centroids import centroid_2dg, centroid_com, centroid_1dg
from scipy.ndimage import shift

# Make images non-blurry on high pixel density screens
%config InlineBackend.figure_format = 'retina'

# Title size
plt.rcParams['axes.titlesize'] = 16

# Axes label size
plt.rcParams['axes.labelsize'] = 13


def show_image(image, title):
    """
    Display an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Plot title.
    """
    fig, ax = plt.subplots(figsize=(12, 8))  # Change image size
    plt.rcParams.update({'font.size': 10})  # Change font size
    
    # Scale the image similar to 'zscale' mode in DS9.
    # This makes easier to spot things in the image.
    interval=ZScaleInterval()
    vmin, vmax = interval.get_limits(image)
    norm = ImageNormalize(vmin=vmin, vmax=vmax)
    
    plt.imshow(image, cmap='gray', norm=norm)  # Set color map and pixel scaling
    plt.xlabel('x [pixel]')  # Set axis labels
    plt.ylabel('y [pixel]')
    plt.title(title, y=-0.2)  # Set image title
    plt.colorbar()  # Show color bar
    

def print_image_stats(image, title):
    """
    Print first pixel value, average and standard deviation for an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Image name.
    """
    
    data = np.asarray(image)# Get numpy array for image data
    label_len = 10  # Length of the text label
    first_pixel = data[0, 0]  # First pixel
    average = np.mean(data)  # Average
    standard_deviation = np.std(data)   # Standard deviation


    # Print values
    # -------

    print(
        f'\n{title}',
        f"\n{'-' * len(title)}",
        f"\n{'Pixel:':<10}{first_pixel:>10.2f} ADU",
        f"\n{'Avg:':<10}{average:>10.2f} ADU",
        f"\n{'Std:':<10}{standard_deviation:>10.2f} ADU\n"
    )
    

def save_image(image, file_path):
    """
    Save image to disk. Overwrites the file if it already exist.
    
    Parameters
    ---------
    image: astropy.nddata.ccddata.CCDData
        Image to be saved
        
    file_path: str
        Path where the image is saved
    """
    
    # Delete the file if it already exists
    
    try:
        os.remove(file_path)
    except OSError:
        pass
    
    # Create directory
    # ------

    dirname = os.path.dirname(file_path)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    image.write(file_path)

## Selecting reference stars

I have choosen four reference stars, shown in Table 1. I have chosen the stars using the following method:

* In DS9, opened images for all filters from photometric night, located in code/050_scaling_and_combining/march_29_2018_stacked directory and well as code/040_shift/data/shifted/march_29_2018/NGC_3201_B_30.000secs_00000472.fit image.

* Chose Zoom > Invert Y in DS9.

* In DS9, opened images for all filters from non-photometric night, located in code/050_scaling_and_combining/march_09_2018_stacked directory. 

* Chose Zoom > Invert X in DS9. Now all FITS images are in same orientation. The images also almost match orientation on Aladin Lite web site: our fits files are about rotated about 3 degrees clockwise with respect to Aladin.

* First I use the B-filter photometric image and look at stars that are visible on it (meaning they have significant blue component).

* Then, I locate those stars on other FITS images and make sure they are not oversaturated, meaning that the radius is not larger than 10 pixels. In addition, I make sure there are no other bright stars closer than 20 pixels.

Table 1: Reference stars chosen manually from photomoteric (March 29 2018), non-photometric (March 9 2018) images. The RA/DEC coordinates and B,V magnitudes are taken from SIMBAD database using Alain Lite web site.

| x, y position, March 9 2018 | x, y position, March 29 2018 | RAJ2000 [deg]  | DEJ2000 [deg]  | B      | V      |
|:---------------------------:|:----------------------------:|----------------|----------------|--------|--------|
|           739, 468          |            49, 28            | 154.3111238862 | -46.4466457933 | 14.730 | 13.800 |

Measure fluxes of reference stars using the following equations (supplied to us by teachers):

$B = 19.237 - 2.5 \log(f/t) - 0.330*A \tag{1}$
$V = 19.696 - 2.5 \log(f/t) - 0.210*A \tag{2}$
$R = 19.679 - 2.5 \log(f/t) - 0.119*A \tag{3}$
$I = 19.079 - 2.5 \log(f/t) - 0.134*A \tag{4},$

where

* $A$ is the airmass number, taken from AIRMASS value from header of the FITS file,
* $f$ is measured flux in ADU,
* $t$ is exposure time of the frame, in seconds.

My first task is to select found bright blue stars that are present and not oversaturated in both photomteric